In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
df = pd.read_json('/data/raw/val_data.json', lines=True)

In [34]:
df.head()

,image_url,image_path,brand,product_title,class_label,color
0,https://m.media-amazon.com/images/I/81LOPbFPiQ...,images/val/0.jpeg,Generic,Women's Khadi Cotton Saree With Blouse Piece (...,saree,NaN
1,https://m.media-amazon.com/images/I/81Q8Oktw4s...,images/val/1.jpeg,Yashvi Designer,Women's Net Saree With Unstitched Blouse Piece,saree,NaN
2,https://m.media-amazon.com/images/I/613S1YGCmo...,images/val/2.jpeg,Aarrah,Georgette Strip Print Saree[S_SHIKHA30021SR02_...,saree,NaN
3,https://m.media-amazon.com/images/I/819budhQl1...,images/val/3.jpeg,MIMOSA,Women's Patola Style Art Silk Saree (Green),saree,NaN
4,https://m.media-amazon.com/images/I/61cAZ94ZQV...,images/val/4.jpeg,Generic,Women's Pure Hand Block Patola Printed Cotton ...,saree,NaN


In [35]:
df.tail()

,image_url,image_path,brand,product_title,class_label,color
7495,https://m.media-amazon.com/images/I/614GvuaIJu...,images/val/7495.jpeg,ishin,Women's Rayon Pink Printed A-Line Kurta Palazz...,women_kurta,NaN
7496,https://m.media-amazon.com/images/I/81WAbf4-ao...,images/val/7496.jpeg,Cotton Culture,Women's Kesar Pink Cotton Straight Kurta,women_kurta,NaN
7497,https://m.media-amazon.com/images/I/61unbmOA7k...,images/val/7497.jpeg,Bae's Wardrobe,Printed Rayon Anarkali Kurti with Plazzo Pant ...,women_kurta,NaN
7498,https://m.media-amazon.com/images/I/71zULAnVNY...,images/val/7498.jpeg,Aarika,Girl's Regular Dress,women_kurta,NaN
7499,https://m.media-amazon.com/images/I/71mhB7xFHt...,images/val/7499.jpeg,Aurelia,Women's Straight Kurta,women_kurta,NaN


In [36]:
df.shape

(7500, 6)

In [37]:
df.class_label.unique()

array(['saree', 'blouse', 'dhoti_pants', 'dupattas', 'gowns', 'kurta_men',
       'leggings_and_salwars', 'lehenga', 'mojaris_men', 'mojaris_women',
       'nehru_jackets', 'palazzos', 'petticoats', 'sherwanis',
       'women_kurta'], dtype=object)

In [38]:
df_revised = df.copy()

In [39]:
df_revised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   image_url      7500 non-null   object
 1   image_path     7500 non-null   object
 2   brand          7500 non-null   object
 3   product_title  7500 non-null   object
 4   class_label    7500 non-null   object
 5   color          947 non-null    object
dtypes: object(6)
memory usage: 351.7+ KB


In [40]:
df_revised = df_revised.reset_index(names='number')

In [41]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color
0,0,https://m.media-amazon.com/images/I/81LOPbFPiQ...,images/val/0.jpeg,Generic,Women's Khadi Cotton Saree With Blouse Piece (...,saree,NaN
1,1,https://m.media-amazon.com/images/I/81Q8Oktw4s...,images/val/1.jpeg,Yashvi Designer,Women's Net Saree With Unstitched Blouse Piece,saree,NaN
2,2,https://m.media-amazon.com/images/I/613S1YGCmo...,images/val/2.jpeg,Aarrah,Georgette Strip Print Saree[S_SHIKHA30021SR02_...,saree,NaN
3,3,https://m.media-amazon.com/images/I/819budhQl1...,images/val/3.jpeg,MIMOSA,Women's Patola Style Art Silk Saree (Green),saree,NaN
4,4,https://m.media-amazon.com/images/I/61cAZ94ZQV...,images/val/4.jpeg,Generic,Women's Pure Hand Block Patola Printed Cotton ...,saree,NaN


In [42]:
df_revised.tail()

,number,image_url,image_path,brand,product_title,class_label,color
7495,7495,https://m.media-amazon.com/images/I/614GvuaIJu...,images/val/7495.jpeg,ishin,Women's Rayon Pink Printed A-Line Kurta Palazz...,women_kurta,NaN
7496,7496,https://m.media-amazon.com/images/I/81WAbf4-ao...,images/val/7496.jpeg,Cotton Culture,Women's Kesar Pink Cotton Straight Kurta,women_kurta,NaN
7497,7497,https://m.media-amazon.com/images/I/61unbmOA7k...,images/val/7497.jpeg,Bae's Wardrobe,Printed Rayon Anarkali Kurti with Plazzo Pant ...,women_kurta,NaN
7498,7498,https://m.media-amazon.com/images/I/71zULAnVNY...,images/val/7498.jpeg,Aarika,Girl's Regular Dress,women_kurta,NaN
7499,7499,https://m.media-amazon.com/images/I/71mhB7xFHt...,images/val/7499.jpeg,Aurelia,Women's Straight Kurta,women_kurta,NaN


In [43]:
columns = ['kurta', 'women', 'men', 'dhoti', 'saree', 'sherwani', 'palazzo', 
           'mojari', 'nehru', 'legging', 'salwar', 'petticoat', 'blouse', 
           'dupatta', 'gown','lehenga']


In [44]:
for column in columns:
    df_revised[str(column)] = df_revised.product_title.str.contains(str(column), flags=re.IGNORECASE)

In [45]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,women,men,...,palazzo,mojari,nehru,legging,salwar,petticoat,blouse,dupatta,gown,lehenga
0,0,https://m.media-amazon.com/images/I/81LOPbFPiQ...,images/val/0.jpeg,Generic,Women's Khadi Cotton Saree With Blouse Piece (...,saree,NaN,False,True,True,...,False,False,False,False,False,False,True,False,False,False
1,1,https://m.media-amazon.com/images/I/81Q8Oktw4s...,images/val/1.jpeg,Yashvi Designer,Women's Net Saree With Unstitched Blouse Piece,saree,NaN,False,True,True,...,False,False,False,False,False,False,True,False,False,False
2,2,https://m.media-amazon.com/images/I/613S1YGCmo...,images/val/2.jpeg,Aarrah,Georgette Strip Print Saree[S_SHIKHA30021SR02_...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/819budhQl1...,images/val/3.jpeg,MIMOSA,Women's Patola Style Art Silk Saree (Green),saree,NaN,False,True,True,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/61cAZ94ZQV...,images/val/4.jpeg,Generic,Women's Pure Hand Block Patola Printed Cotton ...,saree,NaN,False,True,True,...,False,False,False,False,False,False,True,False,False,False


In [46]:
df_revised.shape

(7500, 23)

In [49]:
df_revised_men = df_revised[df_revised.men == True]
df_revised_men.shape

(5502, 23)

In [50]:
df_revised_men.loc[0]

number                                                           0
image_url        https://m.media-amazon.com/images/I/81LOPbFPiQ...
image_path                                       images/val/0.jpeg
brand                                                      Generic
product_title    Women's Khadi Cotton Saree With Blouse Piece (...
class_label                                                  saree
color                                                          NaN
kurta                                                        False
women                                                         True
men                                                           True
dhoti                                                        False
saree                                                         True
sherwani                                                     False
palazzo                                                      False
mojari                                                       F

In [51]:
df_revised.columns

Index(['number', 'image_url', 'image_path', 'brand', 'product_title',
       'class_label', 'color', 'kurta', 'women', 'men', 'dhoti', 'saree',
       'sherwani', 'palazzo', 'mojari', 'nehru', 'legging', 'salwar',
       'petticoat', 'blouse', 'dupatta', 'gown', 'lehenga'],
      dtype='object')

In [52]:
c1 = (df_revised['men'] == True) 
c2 = (df_revised['women'] == True)

df_revised.loc[c1 & c2, 'men'] = False

In [53]:
df_revised_men = df_revised[df_revised.men == True]

In [54]:
df_revised_men.shape

(1594, 23)

In [55]:
df_revised_men.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,women,men,...,palazzo,mojari,nehru,legging,salwar,petticoat,blouse,dupatta,gown,lehenga
1002,1002,"https://assets.myntassets.com/h_720,q_90,w_540...",images/val/1002.jpeg,VASTRAMAY,Men Black Solid Dhoti,dhoti_pants,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1013,1013,https://m.media-amazon.com/images/I/61GA6b9scm...,images/val/1013.jpeg,Larwa,Men's Embroidery Kurta & Salwar Dhoti Set Spec...,dhoti_pants,NaN,True,False,True,...,False,False,False,False,True,False,False,False,False,False
1016,1016,"https://assets.myntassets.com/h_720,q_90,w_540...",images/val/1016.jpeg,Nakshi,Men Black &amp; Gold-coloured Hand Block Print...,dhoti_pants,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1021,1021,https://rukminim1.flixcart.com/image/309/371/k...,images/val/1021.jpeg,N/A,GREEN TEE Solid Men Dhoti,dhoti_pants,N/A,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1023,1023,https://rukminim1.flixcart.com/image/309/371/j...,images/val/1023.jpeg,N/A,GREEN TEE Solid Men Dhoti,dhoti_pants,N/A,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [56]:
df_revised.iloc[2,4]

'Georgette Strip Print Saree[S_SHIKHA30021SR02_perent_Red & Yellow]'

In [57]:
df_revised.iloc[1,4]

"Women's Net Saree With Unstitched Blouse Piece"

In [58]:
df_revised.loc[1000]

number                                                        1000
image_url        https://rukminim1.flixcart.com/image/309/371/k...
image_path                                    images/val/1000.jpeg
brand                                                          N/A
product_title    MK Fashion Hub Red Rayon Fabric Floral Print W...
class_label                                            dhoti_pants
color                                                          N/A
kurta                                                        False
women                                                         True
men                                                          False
dhoti                                                         True
saree                                                        False
sherwani                                                     False
palazzo                                                      False
mojari                                                       F

In [59]:
df_revised.iloc[1000,4]

'MK Fashion Hub Red Rayon Fabric Floral Print Women Dhoti'

In [60]:
df_revised['women_kurta'] = [False] * 7500

In [61]:
df_revised['kurta_men'] = [False] * 7500

In [62]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,women,men,...,nehru,legging,salwar,petticoat,blouse,dupatta,gown,lehenga,women_kurta,kurta_men
0,0,https://m.media-amazon.com/images/I/81LOPbFPiQ...,images/val/0.jpeg,Generic,Women's Khadi Cotton Saree With Blouse Piece (...,saree,NaN,False,True,False,...,False,False,False,False,True,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/81Q8Oktw4s...,images/val/1.jpeg,Yashvi Designer,Women's Net Saree With Unstitched Blouse Piece,saree,NaN,False,True,False,...,False,False,False,False,True,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/613S1YGCmo...,images/val/2.jpeg,Aarrah,Georgette Strip Print Saree[S_SHIKHA30021SR02_...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/819budhQl1...,images/val/3.jpeg,MIMOSA,Women's Patola Style Art Silk Saree (Green),saree,NaN,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/61cAZ94ZQV...,images/val/4.jpeg,Generic,Women's Pure Hand Block Patola Printed Cotton ...,saree,NaN,False,True,False,...,False,False,False,False,True,False,False,False,False,False


In [63]:
c1 = (df_revised['kurta'] == True) 
c2 = (df_revised['women'] == True)

df_revised.loc[c1 & c2, 'women_kurta'] = True

In [64]:
df = df_revised[df_revised.women_kurta == True]

In [65]:
df.shape

(500, 25)

In [66]:
c1 = (df_revised['kurta'] == True) 
c2 = (df_revised['men'] == True)


df_revised.loc[c1 & c2, 'kurta_men'] = True

In [67]:
df_kurta = df_revised[df_revised.kurta == True]

In [68]:
df_kurta1 = df_kurta[df_kurta.kurta_men == False]

In [69]:
df_kurta2 = df_kurta1[df_kurta1.women_kurta == False]

In [70]:
df_kurta2.shape

(66, 25)

In [71]:
df_kurta2_men = df_kurta2[df_kurta2.class_label == 'kurta_men']
df_kurta2_men.shape

(21, 25)

In [72]:
df_kurta2_women = df_kurta2[df_kurta2.class_label == 'women_kurta']
df_kurta2_women.shape

(26, 25)

In [74]:
df_revised['mojaris_men'] = [False] * 7500
df_revised['mojaris_women'] = [False] * 7500
df_revised['leggings_and_salwars'] = [False] * 7500

In [75]:
con1 = (df_revised['mojari'] == True) 
con2 = (df_revised['women'] == True)


df_revised.loc[con1 & con2, 'mojaris_women'] = True

In [76]:
con1 = (df_revised['mojari'] == True) 
con2 = (df_revised['men'] == True)


df_revised.loc[con1 & con2, 'mojaris_men'] = True

In [77]:
df_revised[str(column)] = df_revised.product_title.str.contains(str(column), flags=re.IGNORECASE)

In [78]:
con1 = (df_revised['legging'] == True) 
con2 = (df_revised['salwar'] == True)


df_revised.loc[con1 | con2, 'leggings_and_salwars'] = True

In [79]:
df_revised.leggings_and_salwars.nunique()

2

In [80]:
df_revised.drop(['men', 'women', 'legging','salwar','mojari'], axis=1, inplace=True)

In [81]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti,saree,...,petticoat,blouse,dupatta,gown,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
0,0,https://m.media-amazon.com/images/I/81LOPbFPiQ...,images/val/0.jpeg,Generic,Women's Khadi Cotton Saree With Blouse Piece (...,saree,NaN,False,False,True,...,False,True,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/81Q8Oktw4s...,images/val/1.jpeg,Yashvi Designer,Women's Net Saree With Unstitched Blouse Piece,saree,NaN,False,False,True,...,False,True,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/613S1YGCmo...,images/val/2.jpeg,Aarrah,Georgette Strip Print Saree[S_SHIKHA30021SR02_...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/819budhQl1...,images/val/3.jpeg,MIMOSA,Women's Patola Style Art Silk Saree (Green),saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/61cAZ94ZQV...,images/val/4.jpeg,Generic,Women's Pure Hand Block Patola Printed Cotton ...,saree,NaN,False,False,True,...,False,True,False,False,False,False,False,False,False,False


In [82]:
# Removing the blouse label from all sarees in class_label because 
# models wearing sarees cannot be shown without a saree blouse

c1 = (df_revised['class_label'] == 'saree') 
c2 = (df_revised['blouse'] == True)

df_revised.loc[c1 & c2, 'blouse'] = False

In [83]:
# Removing the saree label from all blouses in class_label because 
# blouses are blouses for sarees or lehengas. Therefore, it is likely that saree 
# will be in the product description

c1 = (df_revised['class_label'] == 'blouse') 
c2 = (df_revised['saree'] == True)

df_revised.loc[c1 & c2, 'saree'] = False

In [84]:
# Removing the saree label from all blouses in class_label because 
# blouses are blouses for sarees or lehengas.Therefore, it is likely that lehenga 
# will be in the product description

c1 = (df_revised['class_label'] == 'blouse') 
c2 = (df_revised['lehenga'] == True)

df_revised.loc[c1 & c2, 'lehenga'] = False

In [85]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti,saree,...,petticoat,blouse,dupatta,gown,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
0,0,https://m.media-amazon.com/images/I/81LOPbFPiQ...,images/val/0.jpeg,Generic,Women's Khadi Cotton Saree With Blouse Piece (...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/81Q8Oktw4s...,images/val/1.jpeg,Yashvi Designer,Women's Net Saree With Unstitched Blouse Piece,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/613S1YGCmo...,images/val/2.jpeg,Aarrah,Georgette Strip Print Saree[S_SHIKHA30021SR02_...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/819budhQl1...,images/val/3.jpeg,MIMOSA,Women's Patola Style Art Silk Saree (Green),saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/61cAZ94ZQV...,images/val/4.jpeg,Generic,Women's Pure Hand Block Patola Printed Cotton ...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [86]:
df_revised.loc[1]

number                                                                  1
image_url               https://m.media-amazon.com/images/I/81Q8Oktw4s...
image_path                                              images/val/1.jpeg
brand                                                     Yashvi Designer
product_title              Women's Net Saree With Unstitched Blouse Piece
class_label                                                         saree
color                                                                 NaN
kurta                                                               False
dhoti                                                               False
saree                                                                True
sherwani                                                            False
palazzo                                                             False
nehru                                                               False
petticoat                             

In [87]:
df_revised = df_revised.rename({'dhoti':'dhoti_pants', 'sherwani':'sherwanis', 'palazzo':'palazzos', 
             'nehru':'nehru_jackets', 'petticoat':'petticoats', 'dupatta':'dupattas', 
             'gown':'gowns'},axis=1)
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
0,0,https://m.media-amazon.com/images/I/81LOPbFPiQ...,images/val/0.jpeg,Generic,Women's Khadi Cotton Saree With Blouse Piece (...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/81Q8Oktw4s...,images/val/1.jpeg,Yashvi Designer,Women's Net Saree With Unstitched Blouse Piece,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/613S1YGCmo...,images/val/2.jpeg,Aarrah,Georgette Strip Print Saree[S_SHIKHA30021SR02_...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/819budhQl1...,images/val/3.jpeg,MIMOSA,Women's Patola Style Art Silk Saree (Green),saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/61cAZ94ZQV...,images/val/4.jpeg,Generic,Women's Pure Hand Block Patola Printed Cotton ...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [88]:
# Changing each new column to false if the class_label is the same as the column name

columns = ['dhoti_pants', 'saree', 'sherwanis', 'palazzos', 'nehru_jackets', 'petticoats',
           'blouse','dupattas', 'gowns', 'lehenga', 'women_kurta', 'kurta_men', 
           'mojaris_men', 'mojaris_women', 'leggings_and_salwars']

for column in columns:
    c1 = (df_revised[str(column)] == True)
    c2 = (df_revised['class_label'] == str(column))
    df_revised.loc[c1 & c2, str(column)] = False

df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
0,0,https://m.media-amazon.com/images/I/81LOPbFPiQ...,images/val/0.jpeg,Generic,Women's Khadi Cotton Saree With Blouse Piece (...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/81Q8Oktw4s...,images/val/1.jpeg,Yashvi Designer,Women's Net Saree With Unstitched Blouse Piece,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/613S1YGCmo...,images/val/2.jpeg,Aarrah,Georgette Strip Print Saree[S_SHIKHA30021SR02_...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/819budhQl1...,images/val/3.jpeg,MIMOSA,Women's Patola Style Art Silk Saree (Green),saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/61cAZ94ZQV...,images/val/4.jpeg,Generic,Women's Pure Hand Block Patola Printed Cotton ...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [89]:
def func(x):
    if x == True:
        return str(column)
    else:
        return 0

for column in columns:
    df_revised[str(column)] = df_revised[str(column)].apply(lambda x: func(x))


In [90]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
0,0,https://m.media-amazon.com/images/I/81LOPbFPiQ...,images/val/0.jpeg,Generic,Women's Khadi Cotton Saree With Blouse Piece (...,saree,NaN,False,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,https://m.media-amazon.com/images/I/81Q8Oktw4s...,images/val/1.jpeg,Yashvi Designer,Women's Net Saree With Unstitched Blouse Piece,saree,NaN,False,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,https://m.media-amazon.com/images/I/613S1YGCmo...,images/val/2.jpeg,Aarrah,Georgette Strip Print Saree[S_SHIKHA30021SR02_...,saree,NaN,False,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,https://m.media-amazon.com/images/I/819budhQl1...,images/val/3.jpeg,MIMOSA,Women's Patola Style Art Silk Saree (Green),saree,NaN,False,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,https://m.media-amazon.com/images/I/61cAZ94ZQV...,images/val/4.jpeg,Generic,Women's Pure Hand Block Patola Printed Cotton ...,saree,NaN,False,0,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
df_revised.tail()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
7495,7495,https://m.media-amazon.com/images/I/614GvuaIJu...,images/val/7495.jpeg,ishin,Women's Rayon Pink Printed A-Line Kurta Palazz...,women_kurta,NaN,True,0,0,...,0,0,0,0,0,0,0,0,0,0
7496,7496,https://m.media-amazon.com/images/I/81WAbf4-ao...,images/val/7496.jpeg,Cotton Culture,Women's Kesar Pink Cotton Straight Kurta,women_kurta,NaN,True,0,0,...,0,0,0,0,0,0,0,0,0,0
7497,7497,https://m.media-amazon.com/images/I/61unbmOA7k...,images/val/7497.jpeg,Bae's Wardrobe,Printed Rayon Anarkali Kurti with Plazzo Pant ...,women_kurta,NaN,False,0,0,...,0,0,0,0,0,0,0,0,0,0
7498,7498,https://m.media-amazon.com/images/I/71zULAnVNY...,images/val/7498.jpeg,Aarika,Girl's Regular Dress,women_kurta,NaN,False,0,0,...,0,0,0,0,0,0,0,0,0,0
7499,7499,https://m.media-amazon.com/images/I/71mhB7xFHt...,images/val/7499.jpeg,Aurelia,Women's Straight Kurta,women_kurta,NaN,True,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
df_revised.replace({'kurta': True}, {'kurta': 'kurta'}, inplace=True)

In [93]:
c1 = (df_revised.class_label == 'women_kurta')
c2 = (df_revised.class_label == 'kurta_men')
c3 = (df_revised.kurta == 'kurta')

df_revised.loc[c3 & (c1 | c2), 'kurta'] = 0


In [94]:
df_revised.replace({'kurta': False}, {'kurta': 0}, inplace=True)

In [95]:
df_revised.tail()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
7495,7495,https://m.media-amazon.com/images/I/614GvuaIJu...,images/val/7495.jpeg,ishin,Women's Rayon Pink Printed A-Line Kurta Palazz...,women_kurta,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7496,7496,https://m.media-amazon.com/images/I/81WAbf4-ao...,images/val/7496.jpeg,Cotton Culture,Women's Kesar Pink Cotton Straight Kurta,women_kurta,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7497,7497,https://m.media-amazon.com/images/I/61unbmOA7k...,images/val/7497.jpeg,Bae's Wardrobe,Printed Rayon Anarkali Kurti with Plazzo Pant ...,women_kurta,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7498,7498,https://m.media-amazon.com/images/I/71zULAnVNY...,images/val/7498.jpeg,Aarika,Girl's Regular Dress,women_kurta,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7499,7499,https://m.media-amazon.com/images/I/71mhB7xFHt...,images/val/7499.jpeg,Aurelia,Women's Straight Kurta,women_kurta,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
df_revised.iloc[7498,4]

"Girl's Regular Dress"

In [97]:
df_revised.iloc[1001,4]

'Sweet Threadz Cotton Viscose Lycra Dhoti Patiyala Salwar Harem Bottoms Pants Light Skin Yellow Combo Pack of 2'

In [98]:
df_revised.iloc[1001]

number                                                               1001
image_url               https://m.media-amazon.com/images/I/41MEH6wJ2v...
image_path                                           images/val/1001.jpeg
brand                                                             Generic
product_title           Sweet Threadz Cotton Viscose Lycra Dhoti Patiy...
class_label                                                   dhoti_pants
color                                                                 NaN
kurta                                                                   0
dhoti_pants                                                             0
saree                                                                   0
sherwanis                                                               0
palazzos                                                                0
nehru_jackets                                                           0
petticoats                            

In [99]:
df_revised.gowns.value_counts()

gowns
0        7488
gowns      12
Name: count, dtype: int64

In [100]:
# Gowns are sometimes called dresses

df_revised['dress'] = df_revised.product_title.str.contains('dress', flags=re.IGNORECASE)

In [101]:
c1 = (df_revised.gowns == 0)
c2 = (df_revised.dress == True)
df_revised.loc[c1 & c2, 'gowns'] = 'gowns'

In [102]:
df_revised.gowns.value_counts()

gowns
0        7322
gowns     178
Name: count, dtype: int64

In [103]:
df_revised = df_revised.drop(['number', 'image_url', 'brand',
        'color', 'dress'], axis=1)

In [104]:
df_saree = df_revised[df_revised.saree == 'saree']
df_saree.shape

(183, 19)

In [105]:
df_saree.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1421,images/val/1421.jpeg,Blue Silk Blend Checkes Ready-to-Wear Saree Wi...,dhoti_pants,0,0,saree,0,0,0,0,0,0,0,0,0,0,0,0,0
1994,images/val/1994.jpeg,Lami 9 Meter Maroon Elephant Design Saree Lace...,dupattas,0,0,saree,0,0,0,0,0,0,0,lehenga,0,0,0,0,0
2021,images/val/2021.jpeg,Women's Cotton Green & Red Geometric Print Sar...,gowns,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
2209,images/val/2209.jpeg,Women's Mono Net Embroidered And Pearl Work Sa...,gowns,0,0,saree,0,0,0,0,0,0,0,0,0,0,0,0,0
2340,images/val/2340.jpeg,Anytime Clothing Traditionally Half Saree for ...,gowns,0,0,saree,0,0,0,0,0,0,0,0,0,0,0,0,0


In [106]:
df_saree.iloc[0,1]

'Blue Silk Blend Checkes Ready-to-Wear Saree With Attached pant'

In [107]:
df_blouse = df_revised[df_revised.blouse == 'blouse']
df_blouse.shape

(199, 19)

In [108]:
df_blouse.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1788,images/val/1788.jpeg,Ethnic Yard Woman's Organza Printed Semi-Stitc...,dupattas,0,0,0,0,0,0,0,blouse,0,0,0,0,0,0,0,0
2021,images/val/2021.jpeg,Women's Cotton Green & Red Geometric Print Sar...,gowns,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
2248,images/val/2248.jpeg,Women Floral Printed Georgette Unstiched Fabri...,gowns,0,0,0,0,0,0,0,blouse,0,gowns,0,0,0,0,0,0
2409,images/val/2409.jpeg,Women's Cotton Rust Self Print Saree with Blou...,gowns,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
2453,images/val/2453.jpeg,Women/Girls Unisex Ethnic Unstiched makhmal Do...,gowns,0,0,0,0,0,0,0,blouse,0,gowns,0,0,0,0,0,0


In [109]:
df_blouse.iloc[1,1]

"Women's Cotton Green & Red Geometric Print Saree with Blouse Piece (Sarita-510_Green & Red_Free Size)"

In [110]:
df_blouse.iloc[3,1]

"Women's Cotton Rust Self Print Saree with Blouse Piece (Samaira-604_Rust_Free Size)"

Incorrect label

In [111]:
df_blouse.iloc[4,1]

'Women/Girls Unisex Ethnic Unstiched makhmal Double Shaded Sequin Dress Material for Shirts,Gown, Blouse, garara, etc.'

Incorrect label

In [112]:
df_dhoti_pants = df_revised[df_revised.dhoti_pants == 'dhoti_pants']
df_dhoti_pants.shape

(36, 19)

In [113]:
df_dhoti_pants.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1517,images/val/1517.jpeg,Women's Pink Rayon Straight Short Kurta With D...,dupattas,kurta,dhoti_pants,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
2570,images/val/2570.jpeg,Men White Solid Dhoti Pant,kurta_men,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2674,images/val/2674.jpeg,Men Maroon &amp; Blue Solid Kurta with Dhoti P...,kurta_men,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2789,images/val/2789.jpeg,Men Brown Dupioni Silk Kurta & Beige Dhoti,kurta_men,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2911,images/val/2911.jpeg,Men White Chikankari Embroidered Kurta with Dh...,kurta_men,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [114]:
df_dhoti_pants.iloc[0,1]

"Women's Pink Rayon Straight Short Kurta With Dhoti Pant"

In [115]:
df_dupattas = df_revised[df_revised.dupattas == 'dupattas']
df_dupattas.shape

(279, 19)

In [116]:
df_dupattas.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1020,images/val/1020.jpeg,VCC Rayon Printed Kurti Palazzo & Dupatta Set ...,dhoti_pants,0,0,0,0,palazzos,0,0,0,dupattas,0,0,0,0,0,0,0
1026,images/val/1026.jpeg,Rayon Green Printed Hand Work Kurta Palazzo an...,dhoti_pants,kurta,0,0,0,palazzos,0,0,0,dupattas,0,0,women_kurta,0,0,0,0
1064,images/val/1064.jpeg,Women’s Pure Cotton Plain Semi Patiala Dhoti S...,dhoti_pants,0,0,0,0,0,0,0,0,dupattas,0,0,0,0,0,0,leggings_and_salwars
1076,images/val/1076.jpeg,Women's Pure Cotton Plain Semi Patiala Dhoti S...,dhoti_pants,0,0,0,0,0,0,0,0,dupattas,0,0,0,0,0,0,leggings_and_salwars
1127,images/val/1127.jpeg,Women's Red Poly Silk Kurta With Pant And Dupatta,dhoti_pants,kurta,0,0,0,0,0,0,0,dupattas,0,0,women_kurta,0,0,0,0


In [117]:
df_dupattas.iloc[0,1]

'VCC Rayon Printed Kurti Palazzo & Dupatta Set for Women'

Incorrect class label

In [118]:
df_dupattas.iloc[1,1]

'Rayon Green Printed Hand Work Kurta Palazzo and Dupatta Set for Women'

In [119]:
df_gowns = df_revised[df_revised.gowns == 'gowns']
df_gowns.shape

(178, 19)

In [120]:
df_gowns.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1085,images/val/1085.jpeg,Womens Pure Cotton Multicoloured Checks Off-Wh...,dhoti_pants,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
1482,images/val/1482.jpeg,Girls Western Dresses Top & Floral Print Plazz...,dhoti_pants,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
1538,images/val/1538.jpeg,Women's Georgette Embroidery Dress Material,dupattas,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
1540,images/val/1540.jpeg,Women's Crepe Dress Material,dupattas,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
1559,images/val/1559.jpeg,MBL-3 Top Self Solid Cotton Designer Heavy Exc...,dupattas,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0


In [121]:
df_gowns.iloc[3,1]

"Women's Crepe Dress Material"

Incorrect label

In [122]:
df_kurta = df_revised[df_revised.kurta == 'kurta']
df_kurta.shape

(221, 19)

In [123]:
df_kurta.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1011,images/val/1011.jpeg,Rayon Multi Color Printed Jacket Style Kurta f...,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
1013,images/val/1013.jpeg,Men's Embroidery Kurta & Salwar Dhoti Set Spec...,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,leggings_and_salwars
1015,images/val/1015.jpeg,Women's Regular Kurta,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
1019,images/val/1019.jpeg,Women's Rama Green And Navy Blue Cotton Kurta Set,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
1026,images/val/1026.jpeg,Rayon Green Printed Hand Work Kurta Palazzo an...,dhoti_pants,kurta,0,0,0,palazzos,0,0,0,dupattas,0,0,women_kurta,0,0,0,0


In [124]:
df_kurta.iloc[0,1]

'Rayon Multi Color Printed Jacket Style Kurta for Women'

In [125]:
df_kurta_men = df_revised[df_revised.kurta_men == 'kurta_men']
df_kurta_men.shape

(49, 19)

In [126]:
df_kurta_men.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1013,images/val/1013.jpeg,Men's Embroidery Kurta & Salwar Dhoti Set Spec...,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,leggings_and_salwars
1039,images/val/1039.jpeg,Men's Dupion Silk Regular Kurta and Dhoti Pant...,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0
1174,images/val/1174.jpeg,Men's Silk Blend Kurta Dhoti and Printed Modi ...,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0
1259,images/val/1259.jpeg,"Men's Aqua Cotton Blend Ethnic Jacket, Kurta a...",dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0
1302,images/val/1302.jpeg,Mens Silk Self Design Kurta And Dhoti Pant (Blue),dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0


In [127]:
# Obviously mislabeled

df_kurta_men.iloc[0,1]

"Men's Embroidery Kurta & Salwar Dhoti Set Special for Boys & Men's"

Looks like ncorrect class label

In [128]:
df_leggings_and_salwars = df_revised[df_revised.leggings_and_salwars == 'leggings_and_salwars']
df_leggings_and_salwars.shape

(243, 19)

In [129]:
df_leggings_and_salwars.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1001,images/val/1001.jpeg,Sweet Threadz Cotton Viscose Lycra Dhoti Patiy...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
1004,images/val/1004.jpeg,Areal Fashion Women's Cotton Traditional Patia...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
1006,images/val/1006.jpeg,Women Beige Solid Dhoti Salwar,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
1007,images/val/1007.jpeg,Combo Offer (Pack of 3) Cotton Viscose Lycra D...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
1008,images/val/1008.jpeg,Combo Offer Pack of 3 Cotton Viscose Lycra Dho...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars


In [130]:
df_leggings_and_salwars.iloc[0,1]

'Sweet Threadz Cotton Viscose Lycra Dhoti Patiyala Salwar Harem Bottoms Pants Light Skin Yellow Combo Pack of 2'

In [131]:
df_leggings_and_salwars.iloc[4,1]

'Combo Offer Pack of 3 Cotton Viscose Lycra Dhoti Patiyala Salwar Harem Bottoms Pants for Womens White Dark Skin Baby Pink'

Incorrect label

In [132]:
df_lehenga = df_revised[df_revised.lehenga == 'lehenga']
df_lehenga.shape

(22, 19)

In [133]:
df_lehenga.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
182,images/val/182.jpeg,Women' Navy Blue Colour Heavy Work Sleeve Blou...,saree,0,0,0,0,0,0,0,0,0,0,lehenga,0,0,0,0,0
1686,images/val/1686.jpeg,Chanderi Digital Printed Latest Design Dupatta...,dupattas,kurta,0,0,0,0,0,0,0,0,0,lehenga,women_kurta,0,0,0,0
1783,images/val/1783.jpeg,Women's Silk Semi-stitched Lehenga Choli (pala...,dupattas,0,0,0,0,0,0,0,0,0,0,lehenga,0,0,0,0,0
1799,images/val/1799.jpeg,Women's Art Silk Semi-stitched Lehenga Choli w...,dupattas,0,0,0,0,0,0,0,0,0,0,lehenga,0,0,0,0,0
1810,images/val/1810.jpeg,Women's Net Semi-stitched Lehenga Choli With D...,dupattas,0,0,0,0,0,0,0,0,0,0,lehenga,0,0,0,0,0


In [134]:
df_lehenga.iloc[0,1]

"Women' Navy Blue Colour Heavy Work Sleeve Blouse For Saree And Lehenga (Blue,Size 38+margin)"

In [135]:
df_mojaris_men = df_revised[df_revised.mojaris_men == 'mojaris_men']
df_mojaris_men.shape

(17, 19)

In [136]:
df_mojaris_men.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
4530,images/val/4530.jpeg,Men's Ethnic Mojari/Juttis/Nagra/Loafers,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
4627,images/val/4627.jpeg,Wedding Mojaris Shoes for Men Loafer Stylish B...,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
4631,images/val/4631.jpeg,Pathani Mojaris for Men,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
4665,images/val/4665.jpeg,"Men's Premium Brown Traditional Leather Jutti,...",mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
4677,images/val/4677.jpeg,Men Casual Designer Shoe-Style Ethnic and Trad...,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0


Many of the images labeled mojaris_women are actually men's

In [137]:
df_mojaris_women = df_revised[df_revised.mojaris_women == 'mojaris_women']
df_mojaris_women.shape

(3, 19)

In [138]:
df_mojaris_women.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
4197,images/val/4197.jpeg,Rahi Fasion Designer Embroidery Work Velvet Ja...,mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0
4394,images/val/4394.jpeg,Ladies Jutti Women Jutti Women Mojari Brown,mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0
4484,images/val/4484.jpeg,Women's Mojaris,mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0


In [142]:
df_mojaris_women.iloc[0,1]

'Rahi Fasion Designer Embroidery Work Velvet Jaipuri Mojari for Women and Girls'

These are actually women's mojaris

In [143]:
df_nehru_jackets = df_revised[df_revised.nehru_jackets == 'nehru_jackets']
df_nehru_jackets.shape

(7, 19)

In [144]:
df_nehru_jackets.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
545,images/val/545.jpeg,Kullu Karishma Hand Woven Merino wool Sleevele...,blouse,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0
2536,images/val/2536.jpeg,Men Beige Solid Kurta with Trousers &amp; Nehr...,kurta_men,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0
2641,images/val/2641.jpeg,Men White &amp; Green Solid Kurta with Churida...,kurta_men,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0
2744,images/val/2744.jpeg,Men Grey &amp; Golden Solid Kurta with Churida...,kurta_men,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0
2798,images/val/2798.jpeg,Men Orange Solid Kurta with Churidar &amp; Neh...,kurta_men,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0


In [145]:
df_nehru_jackets.iloc[0,1]

'Kullu Karishma Hand Woven Merino wool Sleeveless Nehru Jacket Waistcoat'

In [146]:
df_palazzos = df_revised[df_revised.palazzos == 'palazzos']
df_palazzos.shape

(203, 19)

In [147]:
df_palazzos.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1020,images/val/1020.jpeg,VCC Rayon Printed Kurti Palazzo & Dupatta Set ...,dhoti_pants,0,0,0,0,palazzos,0,0,0,dupattas,0,0,0,0,0,0,0
1026,images/val/1026.jpeg,Rayon Green Printed Hand Work Kurta Palazzo an...,dhoti_pants,kurta,0,0,0,palazzos,0,0,0,dupattas,0,0,women_kurta,0,0,0,0
1045,images/val/1045.jpeg,Women's Palazzo Bottom,dhoti_pants,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0
1047,images/val/1047.jpeg,"Women's Regular Fit Full Length, Design for Mu...",dhoti_pants,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0
1051,images/val/1051.jpeg,"Women's Regular Fit Full Length, Design for Mu...",dhoti_pants,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0


In [148]:
df_palazzos.iloc[0,1]

'VCC Rayon Printed Kurti Palazzo & Dupatta Set for Women'

In [149]:
df_palazzos.iloc[2,1]

"Women's Palazzo Bottom"

There are a number of these that should be labeled palazzos.

In [150]:
df_petticoats = df_revised[df_revised.petticoats == 'petticoats']
df_petticoats.shape

(1, 19)

In [151]:
df_petticoats.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
30,images/val/30.jpeg,Women's Pure Cotton Readymade Inskirt Saree Pe...,saree,0,0,0,0,0,0,petticoats,0,0,0,0,0,0,0,0,0


In [152]:
df_petticoats.iloc[0,1]

"Women's Pure Cotton Readymade Inskirt Saree Petticoats Combo with Handmade Nada (Light Sky Blue)"

Mislabeled

In [154]:
df_sherwanis = df_revised[df_revised.sherwanis == 'sherwanis']
df_sherwanis.shape

(44, 19)

In [155]:
df_sherwanis.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1288,images/val/1288.jpeg,Boy's Blended Sherwani and Breeches Set,dhoti_pants,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0
4016,images/val/4016.jpeg,"Men's Traditional Wedding and Sherwani Jutti, ...",mojaris_men,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0
4043,images/val/4043.jpeg,Mens Sherwani Jutti Tradition Ethnic Wedding I...,mojaris_men,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0
4051,images/val/4051.jpeg,"Men's Traditional Black Wedding, Ethnic, Party...",mojaris_men,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0
4056,images/val/4056.jpeg,"Men's Red Wedding, Party & Sherwani Mojaris (J...",mojaris_men,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0


In [156]:
df_sherwanis.iloc[3,1]

"Men's Traditional Black Wedding, Ethnic, Party and Sherwani Jutti,Mojari"

In [157]:
df_women_kurta = df_revised[df_revised.women_kurta == 'women_kurta']
df_women_kurta.shape

(153, 19)

In [158]:
df_women_kurta.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1011,images/val/1011.jpeg,Rayon Multi Color Printed Jacket Style Kurta f...,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
1015,images/val/1015.jpeg,Women's Regular Kurta,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
1019,images/val/1019.jpeg,Women's Rama Green And Navy Blue Cotton Kurta Set,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
1026,images/val/1026.jpeg,Rayon Green Printed Hand Work Kurta Palazzo an...,dhoti_pants,kurta,0,0,0,palazzos,0,0,0,dupattas,0,0,women_kurta,0,0,0,0
1104,images/val/1104.jpeg,Star Fashions Women's Rayon A-Line Kurta & Dho...,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,leggings_and_salwars


In [159]:
df_women_kurta.iloc[0,1]

'Rayon Multi Color Printed Jacket Style Kurta for Women'

In [117]:
df_women_kurta.iloc[1,1]

"Women's Crepe Straight Kurta"

A number of these are mislabeled

In [160]:
df_revised.to_csv('/data/interim/train_other_categories.csv')